# Install Dependency

In [2]:
!pip install beautifulsoup4
!pip install pandas

Note: you may need to restart the kernel to use updated packages.


# Import Packages

In [3]:
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd

# Extract Data from HTML to CSV

In [4]:
# Load the HTML file
with open("/kaggle/input/dataset-name/My Activity.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Find all divs with class "content-cell"
content_cells = soup.find_all('div', class_='content-cell')

# Extract and store transactions
data_rows = []
for cell in content_cells:
    # Extract all text properly
    text_lines = [text.strip() for text in cell.stripped_strings]

    # Check if the first line contains a valid transaction type
    if len(text_lines) >= 2 and any(keyword in text_lines[0] for keyword in ["Paid", "Sent", "Received"]):
        transaction_text = text_lines[0]  # First line is the transaction detail
        date_text = " ".join(text_lines[1:])  # Join all remaining lines for date
        data_rows.append([transaction_text, date_text])

# Write to CSV
csv_file = "transactions.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Transaction", "Date"])  # Header row
    writer.writerows(data_rows)

print(f"CSV file '{csv_file}' created with {len(data_rows)} transactions!")

CSV file 'transactions.csv' created with 1702 transactions!


# Transform CSV

In [5]:
# Input and output file names
input_csv = "transactions.csv"
output_csv = "cleaned_transactions.csv"
skipped_csv = "skipped_transactions.csv"  # Debug file for skipped transactions

# Regex patterns:
pattern_paid = re.compile(r"Paid\s+[^\d]*([\d,]+(?:\.\d+)?)\s+to\s+(.+?)(?:\s+using|$)")
pattern_paid_no_to = re.compile(r"Paid\s+[^\d]*([\d,]+(?:\.\d+)?)\s+using")
pattern_paid_amount_only = re.compile(r"Paid\s+[^\d]*([\d,]+(?:\.\d+)?)$")

pattern_sent = re.compile(r"Sent\s+[^\d]*([\d,]+(?:\.\d+)?)\s+to\s+(.+?)(?:\s+using|$)")
pattern_sent_no_to = re.compile(r"Sent\s+[^\d]*([\d,]+(?:\.\d+)?)\s+using")  # Handles missing recipient
pattern_sent_amount_only = re.compile(r"Sent\s+[^\d]*([\d,]+(?:\.\d+)?)$")  # Sent ₹100.00

pattern_received = re.compile(r"Received\s+[^\d]*([\d,]+(?:\.\d+)?)\s+from\s+(.+?)(?:\s+using|$)")
pattern_received_no_from = re.compile(r"Received\s+[^\d]*([\d,]+(?:\.\d+)?)")

# Read and process the CSV
data_rows = []
skipped_rows = []
with open(input_csv, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = next(reader)  # Skip the header row

    for row in reader:
        if len(row) < 2:
            skipped_rows.append(row)  # If row is incomplete, log it
            continue

        transaction_text, date_text = row  # Extract columns

        # Match different transaction types
        match_paid = pattern_paid.search(transaction_text)
        match_paid_no_to = pattern_paid_no_to.search(transaction_text)
        match_paid_amount_only = pattern_paid_amount_only.search(transaction_text)
        match_sent = pattern_sent.search(transaction_text)
        match_sent_no_to = pattern_sent_no_to.search(transaction_text)
        match_sent_amount_only = pattern_sent_amount_only.search(transaction_text)
        match_received = pattern_received.search(transaction_text)
        match_received_no_from = pattern_received_no_from.search(transaction_text)

        if match_paid:
            amount = match_paid.group(1).replace(",", "")
            recipient = match_paid.group(2).strip()
            transaction_type = "Debit"
            by_whom = "You"
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        elif match_paid_no_to:
            amount = match_paid_no_to.group(1).replace(",", "")
            recipient = "Unknown"
            transaction_type = "Debit"
            by_whom = "You"
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        elif match_paid_amount_only:
            amount = match_paid_amount_only.group(1).replace(",", "")
            recipient = "Unknown"
            transaction_type = "Debit"
            by_whom = "You"
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        elif match_sent:
            amount = match_sent.group(1).replace(",", "")
            recipient = match_sent.group(2).strip()
            transaction_type = "Debit"
            by_whom = "You"
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        elif match_sent_no_to:
            amount = match_sent_no_to.group(1).replace(",", "")
            recipient = "Unknown"
            transaction_type = "Debit"
            by_whom = "You"
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        elif match_sent_amount_only:
            amount = match_sent_amount_only.group(1).replace(",", "")
            recipient = "Unknown"
            transaction_type = "Debit"
            by_whom = "You"
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        elif match_received:
            amount = match_received.group(1).replace(",", "")
            recipient = "You"
            transaction_type = "Credit"
            by_whom = match_received.group(2).strip()
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        elif match_received_no_from:
            amount = match_received_no_from.group(1).replace(",", "")
            recipient = "You"
            transaction_type = "Credit"
            by_whom = "Unknown"
            data_rows.append([transaction_type, amount, recipient, by_whom, date_text])
        else:
            skipped_rows.append(row)  # Log unmatched transactions

# Write the cleaned data to a new CSV
with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Transaction Type", "Amount", "To/From", "By Whom", "Date"])  # Updated header
    writer.writerows(data_rows)

# Write skipped rows for analysis
if skipped_rows:
    with open(skipped_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Transaction", "Date"])  # Header for skipped rows
        writer.writerows(skipped_rows)

print(f" CSV file '{output_csv}' created successfully with {len(data_rows)} transactions.")
print(f" Skipped {len(skipped_rows)} transactions. Check '{skipped_csv}' for details.")

# Print first 5 skipped rows for quick debugging
if skipped_rows:
    print("\n🔍 Sample Skipped Transactions:")
    for row in skipped_rows[:5]:
        print(row)

 CSV file 'cleaned_transactions.csv' created successfully with 1702 transactions.
 Skipped 0 transactions. Check 'skipped_transactions.csv' for details.


# Load Data and create Montly / Weekly Sheets

In [6]:
# Load the cleaned transactions CSV
input_csv = "cleaned_transactions.csv"
df = pd.read_csv(input_csv)

# Convert 'Date' column to datetime format
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Drop rows where date conversion failed
df = df.dropna(subset=["Date"])

# Convert 'Amount' column to numeric
df["Amount"] = pd.to_numeric(df["Amount"], errors="coerce")

# Create Month and Week columns
df["Month"] = df["Date"].dt.to_period("M")  # Extract YYYY-MM format
df["Week"] = df["Date"].dt.to_period("W")  # Extract YYYY-WW format

# Separate debits (spending) and credits (received)
df_debit = df[df["Transaction Type"] == "Debit"]
df_credit = df[df["Transaction Type"] == "Credit"]

### Calculate Monthly Spending & Received ###
monthly_spending = df_debit.groupby("Month", as_index=False)["Amount"].sum().rename(columns={"Amount": "Amount_Spent"})
monthly_received = df_credit.groupby("Month", as_index=False)["Amount"].sum().rename(columns={"Amount": "Amount_Received"})

# Merge spending and received data for monthly report
monthly_summary = pd.merge(monthly_spending, monthly_received, on="Month", how="outer").fillna(0)

# Calculate Net Amount (Received - Spent)
monthly_summary["Net_Amount"] = monthly_summary["Amount_Received"] - monthly_summary["Amount_Spent"]

# Add a Net Status column to indicate Positive (Surplus) or Negative (Deficit)
monthly_summary["Net_Status"] = monthly_summary["Net_Amount"].apply(lambda x: "Positive" if x >= 0 else "Negative")

monthly_summary.to_csv("monthly_summary.csv", index=False)

### Calculate Weekly Spending & Received ###
weekly_spending = df_debit.groupby("Week", as_index=False)["Amount"].sum().rename(columns={"Amount": "Amount_Spent"})
weekly_received = df_credit.groupby("Week", as_index=False)["Amount"].sum().rename(columns={"Amount": "Amount_Received"})

# Merge spending and received data for weekly report
weekly_summary = pd.merge(weekly_spending, weekly_received, on="Week", how="outer").fillna(0)

# Calculate Net Amount (Received - Spent)
weekly_summary["Net_Amount"] = weekly_summary["Amount_Received"] - weekly_summary["Amount_Spent"]

# Add a Net Status column
weekly_summary["Net_Status"] = weekly_summary["Net_Amount"].apply(lambda x: "Positive" if x >= 0 else "Negative")

weekly_summary.to_csv("weekly_summary.csv", index=False)

print("'monthly_summary.csv' and 'weekly_summary.csv' created successfully!")

'monthly_summary.csv' and 'weekly_summary.csv' created successfully!


<ipython-input-6-4cd6f2f3cea2>:15: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["Month"] = df["Date"].dt.to_period("M")  # Extract YYYY-MM format
<ipython-input-6-4cd6f2f3cea2>:16: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["Week"] = df["Date"].dt.to_period("W")  # Extract YYYY-WW format
